# Task 1

In [1]:
import pandas as pd 
from math import cos, asin, sqrt, isnan
import numpy as np

ports = pd.read_csv("ports.csv")

tracking = pd.read_csv("tracking.csv").drop_duplicates().sort_values(by='datetime', ascending=True)
#.drop_duplicates(subset=['vessel', 'lat', 'long'], keep='first')


In [2]:
# https://stackoverflow.com/a/41337005

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

ports_dict = ports[['lat', 'long']].to_dict('records')

def closest_port(data, v, ports):
    pos = min(data, key=lambda p: distance(v['lat'],v['long'],p['lat'],p['long']))
    dist = distance(pos['lat'], pos['long'], v['lat'], v['long'])
    return (ports[(ports.lat == pos['lat']) & (ports.long == pos['long'])]['port'].values[0], dist)


In [3]:
ports['threshold'] = 10
threshold_30 = [173, 98, 149]
threshold_15 = [38, 96, 72, 51]

ports.loc[ports['port'].isin(threshold_30), 'threshold'] = 30
ports.loc[ports['port'].isin(threshold_15), 'threshold'] = 15

In [4]:
ports.loc[(ports.port==142), 'lat'] = 30.600600
ports.loc[(ports.port==142), 'long'] = 122.104500

ports.loc[(ports.port==76), 'lat'] = 21.903937
ports.loc[(ports.port==76), 'long'] = 113.216557

In [5]:
speed_threshold = 1
ports_dict = ports[['lat', 'long']].to_dict('records')

started_port = 0
voyages_list = []
end_port_id = 0
latest_draft = 0
pd.set_option('display.max_rows', 60)

debug = []
debug_ports = []
special_voyages = []

for vessel in tracking.drop_duplicates(subset=['vessel'])['vessel'].values:
#for vessel in [68]:
    to_add = 0
    for index, row in tracking[tracking.vessel == vessel].iterrows():
        (current_port, closest_dist) = closest_port(ports_dict, {'lat': row['lat'], 'long': row['long']}, ports)
        ports_km_threshold = ports[ports.port == current_port]['threshold'].values[0]
        debug.append({'vessel': vessel, 'datetime': row['datetime'], 'speed': row['speed'], 'draft': row['draft'], 'closest_dist': closest_dist, 'closest_port': current_port, 'lat': row['lat'], 'long': row['long']})
        if started_port == 0 and row['speed'] < speed_threshold:
            #record startport
            begin_port_id = current_port
            if ports_km_threshold > closest_dist:
                started_port = 1
        elif started_port == 1 and row['speed'] >= speed_threshold:
            #record start date 
            started_port = 2
            begin_date = row['datetime']
        elif started_port == 2 and row['speed'] < speed_threshold:
            #record end date, end port, add to new df
            #end_date = row['datetime']
            #end_port_id = current_port
            if ports_km_threshold > closest_dist:
                end_date = row['datetime']
                if (pd.to_datetime(end_date) - pd.to_datetime(begin_date)) / pd.Timedelta(hours=1) >= 24:
                    end_port_id = current_port
                    end_draft = row['draft']
                    voyages_list.append({'vessel': vessel, 'begin_date': begin_date, 'end_date': end_date, 'begin_port_id': begin_port_id, 'end_port_id': end_port_id})
                    started_port = 0
                    if to_add == 1:
                        debug_ports.append(dict_to_add)
                        to_add = 0
                else:
                    started_port = 1
            elif closest_dist <= 100:
                end_date = row['datetime']
                dict_to_add = {'vessel': vessel, 'date': end_date, 'lat': row['lat'], 'long': row['long']}
                to_add = 1
        elif started_port == 0 and row['speed'] >= speed_threshold and end_port_id != 0:
            begin_date = row['datetime']
            begin_port_id = end_port_id
            started_port = 2
        elif started_port == 2 and (abs(latest_draft - row['draft']) > 0.5 or (pd.to_datetime(row['datetime']) - pd.to_datetime(latest_date)) / pd.Timedelta(hours=1) > 12) and latest_dist <= latest_threshold:
        #elif started_port == 2 and (abs(latest_draft - row['draft']) > 0.5 or (pd.Timedelta(pd.to_datetime(row['datetime']) - pd.to_datetime(latest_date)).seconds / 3600.0) > 12) and latest_dist <= ports_km_threshold:
            if ((pd.to_datetime(latest_date) - pd.to_datetime(begin_date)) / pd.Timedelta(hours=1)) > 48:
                voyages_list.append({'vessel': vessel, 'begin_date': begin_date, 'end_date': latest_date, 'begin_port_id': begin_port_id, 'end_port_id': latest_port})
                special_voyages.append({'vessel': vessel, 'begin_date': begin_date, 'end_date': latest_date, 'begin_port_id': begin_port_id, 'end_port_id': latest_port})
                begin_date = row['datetime']
                begin_port_id = latest_port
                started_port = 2
        latest_date = row['datetime']
        latest_draft = row['draft']
        latest_dist = closest_dist
        latest_port = current_port
        latest_threshold = ports_km_threshold
    started_port = 0
    end_port_id = 0
voyages = pd.DataFrame(voyages_list)
debug = pd.DataFrame(debug)
debug_ports = pd.DataFrame(debug_ports)
voyages_dup = voyages[voyages['begin_port_id'] == voyages['end_port_id']]
voyages = voyages[voyages['begin_port_id'] != voyages['end_port_id']]
special_voyages = pd.DataFrame(special_voyages)

voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id
0,131,2019-01-07 07:59:00,2019-02-14 01:57:00,142,48
1,131,2019-02-15 01:59:00,2019-03-02 22:07:00,48,69
2,131,2019-03-04 11:46:00,2019-03-10 01:46:00,69,29
3,131,2019-03-11 06:37:00,2019-03-20 09:05:00,29,69
4,131,2019-03-21 21:54:00,2019-04-05 20:38:46,69,11
...,...,...,...,...,...
3428,107,2019-10-29 23:52:35,2019-11-09 00:49:44,51,99
3429,107,2019-11-10 05:32:07,2019-11-18 18:41:10,99,111
3430,107,2019-11-19 21:54:51,2019-12-01 21:49:30,111,73
3431,107,2019-12-03 00:54:56,2019-12-11 23:21:49,73,72


In [18]:
# Assertions

print(len(voyages[voyages.begin_date > voyages.end_date]) == 0)

found_error = 0
for vessel in voyages.drop_duplicates(subset=['vessel'])['vessel'].values:
    prev_port = 0
    for index, row in voyages[voyages.vessel == vessel].iterrows():
        if prev_port != 0:
            if prev_port != row['begin_port_id']:
                print("Error in index "+ str(index))
                found_error = 1
        prev_port = row['end_port_id']
print(found_error == 0)

True
True


In [19]:
voyages.to_csv('voyages.csv', index=False)


In [19]:
len(voyages.drop_duplicates(subset=['vessel'])['vessel'].values)

172

In [117]:
#list(set(temp1) - set(temp2))
list(set(tracking.drop_duplicates(subset=['vessel'])['vessel'].values) - set(voyages.drop_duplicates(subset=['vessel'])['vessel'].values))

[]